##### **Import packages and set up Neo4**

In [10]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph
load_dotenv()
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [11]:
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [13]:
os.environ['NEO4J_URI']=NEO4J_URI
os.environ['NEO4J_USER']=NEO4J_USERNAME
os.environ['NEO4J_PASSWORD']=NEO4J_PASSWORD

- Initialize a knowledge graph instance using LangChain's Neo4j integration

In [14]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

##### **Querying the movie knowledge graph**
- Match all nodes in the graph

In [18]:
result=graph.query("MATCH (n) RETURN count(n) AS NumberofNodes")
result

[{'NumberofNodes': 180}]

In [19]:
print(f"There are {result[0]['NumberofNodes']} nodes in this graph")

There are 180 nodes in this graph


In [23]:
graph.query("MATCH (people:Person) Return count(people) AS NumberofPeople")

[{'NumberofPeople': 133}]

In [26]:
graph.query("MATCH (tom:Person{name:'Tom Hanks'}) RETURN tom")

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

In [28]:
graph.query("""
MATCH(cloudAtlas:Movie {title:"Cloud Atlas"})
Return cloudAtlas.released,cloudAtlas.tagline
""")

[{'cloudAtlas.released': 2012,
  'cloudAtlas.tagline': 'Everything is connected'}]

**Cypher patterns with conditional matching**

In [29]:
graph.query("""
MATCH (nineties:Movie)
WHERE nineties.released>1990
AND nineties.released<2000
RETURN nineties.title
""")

[{'nineties.title': 'A Few Good Men'},
 {'nineties.title': 'Unforgiven'},
 {'nineties.title': 'Hoffa'},
 {'nineties.title': 'A League of Their Own'},
 {'nineties.title': 'Sleepless in Seattle'},
 {'nineties.title': 'Johnny Mnemonic'},
 {'nineties.title': 'Apollo 13'},
 {'nineties.title': 'That Thing You Do'},
 {'nineties.title': 'The Birdcage'},
 {'nineties.title': 'Twister'},
 {'nineties.title': "The Devil's Advocate"},
 {'nineties.title': 'As Good as It Gets'},
 {'nineties.title': 'What Dreams May Come'},
 {'nineties.title': "You've Got Mail"},
 {'nineties.title': 'When Harry Met Sally'},
 {'nineties.title': 'The Matrix'},
 {'nineties.title': 'Snow Falling on Cedars'},
 {'nineties.title': 'The Green Mile'},
 {'nineties.title': 'Bicentennial Man'}]

**Pattern Matching with Multiple Nodes**

In [30]:
graph.query("""
MATCH (actor:Person) -[:ACTED_IN] ->(movie:Movie)
RETURN actor.name,movie.title LIMIT 10
""")

[{'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix'},
 {'actor.name': 'Emil Eifrem', 'movie.title': 'The Matrix'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Revolutions'}]

In [37]:
graph.query("""
MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies:Movie)
RETURN tom.name,tomHanksMovies.title
""")

[{'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "You've Got Mail"},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Sleepless in Seattle'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Joe Versus the Volcano'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'That Thing You Do'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cloud Atlas'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Da Vinci Code'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Green Mile'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Apollo 13'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cast Away'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "Charlie Wilson's War"},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Polar Express'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'A League of Their Own'}]

In [40]:
graph.query("""
MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors)
RETURN coActors.name,m.title LIMIT 10
""")

[{'coActors.name': 'Meg Ryan', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Greg Kinnear', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Parker Posey', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Dave Chappelle', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Steve Zahn', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Meg Ryan', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Rita Wilson', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Bill Pullman', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Victor Garber', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': "Rosie O'Donnell", 'm.title': 'Sleepless in Seattle'}]

**Adding Data to the Graph**

In [41]:
graph.query("""
CREATE (ashu:Person {name:"ASHU"})
RETURN ashu
""")

[{'ashu': {'name': 'ASHU'}}]

**Adding a Relationship**

In [44]:
graph.query("""
MATCH (ashu:Person {name:"Ashu"}),(shivam:Person {name:"Shivam"})
MERGE (ashu)-[hasRelationship:WORKS_WITH]->(shivam)
RETURN ashu, hasRelationship, shivam
""")

[]

[]